<a href="https://colab.research.google.com/github/rsmani84/Conductometric-Titration-of-simple-acid-Vs-Base/blob/main/ADMET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# ==============================
# 🧬 ADMET PREDICTION TOOL (Colab)
# Single SMILES → Full ADMET Report
# ==============================

# Install (run once in fresh Colab)
!pip install rdkit pandas -q

from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, Crippen, QED
import pandas as pd

# ==============================
# ADMET CALCULATION FUNCTION
# ==============================
def calculate_admet(smiles, name="Compound"):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        return {"Name": name, "SMILES": smiles, "Error": "Invalid SMILES"}

    mw = Descriptors.MolWt(mol)
    logp = Crippen.MolLogP(mol)
    hbd = Lipinski.NumHDonors(mol)
    hba = Lipinski.NumHAcceptors(mol)
    tpsa = Descriptors.TPSA(mol)
    rot = Lipinski.NumRotatableBonds(mol)
    qed = QED.qed(mol)

    # Lipinski Rule
    lipinski_violations = sum([
        mw > 500,
        logp > 5,
        hbd > 5,
        hba > 10
    ])

    # Veber Rule
    veber = "Pass" if (rot <= 10 and tpsa <= 140) else "Fail"

    # Ghose Filter
    ghose = "Pass" if (
        160 <= mw <= 480 and
        -0.4 <= logp <= 5.6 and
        20 <= Descriptors.MolMR(mol) <= 130 and
        40 <= mol.GetNumAtoms() <= 70
    ) else "Fail"

    # Egan Rule
    egan = "Pass" if (tpsa <= 131 and logp <= 5.88) else "Fail"

    # BBB Prediction
    bbb = "BBB+" if (tpsa < 90 and logp > 2) else "BBB-"

    # Solubility proxy
    if logp > 5:
        solubility = "Low"
    elif logp < 0:
        solubility = "High"
    else:
        solubility = "Moderate"

    # Absorption interpretation
    if lipinski_violations == 0:
        absorption = "Good"
    elif lipinski_violations == 1:
        absorption = "Moderate"
    else:
        absorption = "Poor"

    return {
        "Name": name,
        "SMILES": smiles,
        "MW": mw,
        "LogP": logp,
        "HBD": hbd,
        "HBA": hba,
        "TPSA": tpsa,
        "Rotatable Bonds": rot,
        "QED": qed,
        "Lipinski Violations": lipinski_violations,
        "Veber": veber,
        "Ghose": ghose,
        "Egan": egan,
        "BBB": bbb,
        "Solubility": solubility,
        "Absorption": absorption
    }

# ==============================
# USER INPUT
# ==============================
name = input("Enter compound name: ")
smiles = input("Enter SMILES: ")

# ==============================
# RUN ADMET
# ==============================
result = calculate_admet(smiles, name)
admet_df = pd.DataFrame([result])

# Display Table
print("\n📊 ADMET Properties Table")
display(admet_df)

# ==============================
# ADMET SUMMARY
# ==============================
row = admet_df.iloc[0]

print(f"\n🔬 {row['Name']}")
print("Absorption:", row["Absorption"])
print("Bioavailability (Veber):", row["Veber"])
print("Drug-likeness (QED):", round(row["QED"], 3))
print("BBB:", row["BBB"])
print("Solubility:", row["Solubility"])

# ==============================
# SAVE EXCEL REPORT
# ==============================
file_name = f"{name}_ADMET_report.xlsx"
admet_df.to_excel(file_name, index=False)

print(f"\n💾 ADMET report saved as: {file_name}")
from google.colab import files
files.download(file_name)

Enter compound name: rs
Enter SMILES: CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C)NC4=NC=CC(=N4)C5=CN=CC=C5

📊 ADMET Properties Table


,Name,SMILES,MW,LogP,HBD,HBA,TPSA,Rotatable Bonds,QED,Lipinski Violations,Veber,Ghose,Egan,BBB,Solubility,Absorption
0,rs,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,493.615,4.59032,2,7,86.28,7,0.389416,0,Pass,Fail,Pass,BBB+,Moderate,Good



🔬 rs
Absorption: Good
Bioavailability (Veber): Pass
Drug-likeness (QED): 0.389
BBB: BBB+
Solubility: Moderate

💾 ADMET report saved as: rs_ADMET_report.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>